In [2]:
import tatqa_utils
import pandas as pd

In [3]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [4]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2

In [5]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [6]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
#llm = OpenAI(temperature=0) 
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [8]:
def answer(llm, question, table, paragraphs):
    text = "\n\n".join(paragraphs)
    prompt = f"""Find the answer to the following question in the following texts or table! Return the found text spans as Python list and scale in format (value:list, scale:str). Try to delete question from result.  If the answer is not in the text or table return 'None'. \n Question: {question} \n  Text: {text} \n Table: {table}
    Examples:

    Texts : ["Sales by Contract Type: Substantially all of our contracts are fixed-price type contracts. Sales included in Other contract types represent cost plus and time and material type contracts.", "On a fixed-price type contract, we agree to perform the contractual statement of work for a predetermined sales price. On a cost-plus type contract, we are paid our allowable incurred costs plus a profit which can be fixed or variable depending on the contract’s fee arrangement up to predetermined funding levels determined by the customer. On a time-and-material type contract, we are paid on the basis of direct labor hours expended at specified fixed-price hourly rates (that include wages, overhead, allowable general and administrative expenses and profit) and materials at cost. The table below presents total net sales disaggregated by contract type (in millions):"]
    Table : "[['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]"
    
    Question: "What is the company paid on a cost-plus type contract?"
    Response: (["our allowable incurred costs plus a profit which can be fixed or variable depending on the contract’s fee arrangement up to predetermined funding levels determined by the customer"],"")

    Question: "What is the amount of total sales in 2019?"
    Response: (["$1,496.5"], "million")

    Question: "What are the contract types?"
    Response: (["fixed-price type", "cost-plus type", "time-and-material type"],"")

    Question: "In which year is the amount of total sales the largest?"
    Response: (["2019"], "")

    Now, answer the following question:{question}
    Text: {text}
    Table: {table}    
    """
    #prompt = f"Find the answer to the following question in the following texts! Return the result as a Python list. If the answer is not in the texts return 'None'. \n Question: {question} \n Text: {text}"
    #prompt = f"Find the answer to the following question in the following texts! Return the found text spans as a Python list. If the answer is not in the texts return 'None'. \n Question: {question} \n Text: {text}"
    #prompt = f"""
#You are an AI assistant skilled in extracting specific answers from text paragraphs and tables. Provided are some text sections and/or tables containing relevant data. Based on this data, please answer questions accurately and concisely, relying only on the information provided. Structure your response based on the requested answer format, even if multiple options are available.
#Examples:

#Question: "What is the company paid on a cost-plus type contract?"
#Answer: ["our allowable incurred costs plus a profit which can be fixed or variable depending on the contract’s fee arrangement up to predetermined funding levels determined by the customer"]

#Question: "What is the amount of total sales in 2019?"
#Answer: ["$1,496.5"]

#Question: "What are the contract types?"
#Answer: ["fixed-price type", "cost-plus type", "time-and-material type"]

#Question: "In which year is the amount of total sales the largest?"
#Answer: ["2019"]

#Instructions for this task:

#Carefully review the provided text and/or tables to locate the information requested in each question.
#Ensure the answer is complete, accurate, and formatted to match the answer type expected (e.g., single value, list, year).
#When values are numeric, retain units where specified (e.g., "[$1,000.5]" or "[2015]"). Do not write whole sentences.
#ow, answer the following question:{question}
#n Table: {table}\n Text: {text}
#"""    
    gen_txt =  llm.invoke(prompt).content
    #print(gen_txt)
    return (prompt, eval(gen_txt))
    
table = "[['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]"
paragraphs = ["Sales by Contract Type: Substantially all of our contracts are fixed-price type contracts. Sales included in Other contract types represent cost plus and time and material type contracts.", "On a fixed-price type contract, we agree to perform the contractual statement of work for a predetermined sales price. On a cost-plus type contract, we are paid our allowable incurred costs plus a profit which can be fixed or variable depending on the contract’s fee arrangement up to predetermined funding levels determined by the customer. On a time-and-material type contract, we are paid on the basis of direct labor hours expended at specified fixed-price hourly rates (that include wages, overhead, allowable general and administrative expenses and profit) and materials at cost. The table below presents total net sales disaggregated by contract type (in millions):"]
#question="What is the amount of total sales in 2019?"
#question = "What is the company paid on a cost-plus type contract?"
#ans = ["our allowable incurred costs plus a profit which can be fixed or variable depending on the contract’s fee arrangement up to predetermined funding levels determined by the customer"]
#question = "What are the contract types?"

table = "[['', '', 'Fiscal', ''], ['', '2019', '2018', '2017'], ['', '', '(in millions)', ''], ['Transportation Solutions:', '', '', ''], ['Automotive', '$ 5,686', '$ 6,092', '$  5,228'], ['Commercial transportation', '1,221', '1,280', '997'], ['Sensors', '914', '918', '814'], ['Total Transportation Solutions', '7,821', '8,290', '7,039'], ['Industrial Solutions:', '', '', ''], ['Industrial equipment', '1,949', '1,987', '1,747'], ['Aerospace, defense, oil, and gas', '1,306', '1,157', '1,075'], ['Energy', '699', '712', '685'], ['Total Industrial Solutions', '3,954', '3,856', '3,507'], ['Communications Solutions:', '', '', ''], ['Data and devices', '993', '1,068', '963'], ['Appliances', '680', '774', '676'], ['Total Communications Solutions', '1,673', '1,842', '1,639'], ['Total', '$ 13,448', '$ 13,988', '$ 12,185']]"
paragraphs = ["Net sales by segment and industry end market(1) were as follows:", "(1) Industry end market information is presented consistently with our internal management reporting and may be revised periodically as management deems necessary."]
#question = "How is industry end market information presented?"
#question = "In which years was for the net sales by segment and industry end market calculated?"
#question = "What are the types of Solutions segments in the table?"
question = "In which year was the amount for Sensors the largest?"

#question = 'What is the change in Fixed Price in 2019 from 2018?'
#table = "[['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]"
#table = remove_thousand_separators(table)
#print(table)
#question = 'What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?'

res = answer(llm, question, table, paragraphs)
res

('Find the answer to the following question in the following texts or table! Return the found text spans as Python list and scale in format (value:list, scale:str). Try to delete question from result.  If the answer is not in the text or table return \'None\'. \n Question: In which year was the amount for Sensors the largest? \n  Text: Net sales by segment and industry end market(1) were as follows:\n\n(1) Industry end market information is presented consistently with our internal management reporting and may be revised periodically as management deems necessary. \n Table: [[\'\', \'\', \'Fiscal\', \'\'], [\'\', \'2019\', \'2018\', \'2017\'], [\'\', \'\', \'(in millions)\', \'\'], [\'Transportation Solutions:\', \'\', \'\', \'\'], [\'Automotive\', \'$ 5,686\', \'$ 6,092\', \'$  5,228\'], [\'Commercial transportation\', \'1,221\', \'1,280\', \'997\'], [\'Sensors\', \'914\', \'918\', \'814\'], [\'Total Transportation Solutions\', \'7,821\', \'8,290\', \'7,039\'], [\'Industrial Solutions:

In [9]:
t = devdf.iloc[1]['table']['table'];
f"{t}"

"[['', '', 'Fiscal', ''], ['', '2019', '2018', '2017'], ['', '', '(in millions)', ''], ['Transportation Solutions:', '', '', ''], ['Automotive', '$ 5,686', '$ 6,092', '$  5,228'], ['Commercial transportation', '1,221', '1,280', '997'], ['Sensors', '914', '918', '814'], ['Total Transportation Solutions', '7,821', '8,290', '7,039'], ['Industrial Solutions:', '', '', ''], ['Industrial equipment', '1,949', '1,987', '1,747'], ['Aerospace, defense, oil, and gas', '1,306', '1,157', '1,075'], ['Energy', '699', '712', '685'], ['Total Industrial Solutions', '3,954', '3,856', '3,507'], ['Communications Solutions:', '', '', ''], ['Data and devices', '993', '1,068', '963'], ['Appliances', '680', '774', '676'], ['Total Communications Solutions', '1,673', '1,842', '1,639'], ['Total', '$ 13,448', '$ 13,988', '$ 12,185']]"

In [10]:
from tqdm.notebook import tqdm as log_progress

res = []
for i, item in log_progress(devdf.iterrows()):
    table = item['table']['table']
    #table = transform_elements(table, remove_thousand_separators)
    #table = [remove_thousand_separators(cell) for row in table for cell in row]
    #table = [cell.replace('$', '') for row in table for cell in row]
    #print (table)
    paragraphs = [p['text'] for p in item['paragraphs']]
    for q in item['questions']:        
        if 'span' in q['answer_type']: #and 'table' in q['answer_from']:             
            prompt, r  = answer(llm, q, table, paragraphs)
            err =  None
            if r is None:
                (pred_value, pred_unit) = ("", "")
            else:                
                print(r)
                (pred_value, pred_unit) = r
            code = ""   
            #res.append({"table":table, "q":q, "pred":pred, "code": code})
            res.append(({"answer_type":q["answer_type"], "answer": q["answer"], 'scale': q["scale"]}, pred_value, pred_unit, q, code, prompt, item['table'], err))


0it [00:00, ?it/s]

(['our allowable incurred costs plus a profit which can be fixed or variable depending on the contract’s fee arrangement up to predetermined funding levels determined by the customer'], '')
(['$1,496.5'], 'million')
(['fixed-price type', 'cost-plus type', 'time-and-material type'], '')
(['2019'], 'million')
(['Industry end market information is presented consistently with our internal management reporting and may be revised periodically as management deems necessary.'], '')
(['2019', '2018', '2017'], 'list')
(['Transportation Solutions', 'Industrial Solutions', 'Communications Solutions'], 'list')
(['$ 6,092'], 'million')
(['By comparison against the FTSE pension liability index for AA rated corporate instruments'], '')
(['The international discount rates were determined by comparison against country specific AA corporate indices, adjusted for duration of the obligation.'], 'None')
(['The periodic benefit cost and the actuarial present value of projected benefit obligations are based o

KeyboardInterrupt: 

In [12]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_,_ in res[4:]:
    metrics(ans, pred, pred_scale)    
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=False)
print( pred_em, pred_f1, scale_score)

{'answer_type': 'span', 'answer': ['2018'], 'scale': ''}
calc  ['6092000000.0000'] million ['2018.0000']
f1 0.0 
{'answer_type': 'span', 'answer': ['Annual basis'], 'scale': ''}
calc  ['The periodic benefit cost and the actuarial present value of projected benefit obligations are based on actuarial assumptions that are reviewed on an annual basis.']  ['Annual basis']
f1 0.2 
{'answer_type': 'span', 'answer': ['2018'], 'scale': ''}
calc  ['In 2018, IMFT discontinued production of NAND and subsequent to that time manufactured 3D XPoint memory. None'] None ['2018.0000']
f1 0.0 
{'answer_type': 'span', 'answer': ['$1,305'], 'scale': ''}
calc  ['1305000000.0000'] million ['1305.0000']
f1 0.0 
{'answer_type': 'span', 'answer': ['2019'], 'scale': ''}
calc  ['2019000000.0000'] million ['2019.0000']
f1 0.0 
{'answer_type': 'span', 'answer': ['2019'], 'scale': ''}
calc  ['2019000000.0000'] million ['2019.0000']
f1 0.0 
{'answer_type': 'span', 'answer': ['2019'], 'scale': ''}
calc  ['2019000000.0

In [13]:
metrics._details

[{'answer_type': 'span',
  'answer': ['consistently with our internal management reporting and may be revised periodically as management deems necessary.'],
  'scale': '',
  'pred': ['Industry end market information is presented consistently with our internal management reporting and may be revised periodically as management deems necessary.'],
  'pred_scale': '',
  'em': 0.0,
  'f1': 0.82,
  'pred_span': None,
  'gold_span': None,
  'span_em': 0,
  'span_f1': 0},
 {'answer_type': 'multi-span',
  'answer': ['2019', '2018', '2017'],
  'scale': '',
  'pred': ['2019', '2018', '2017'],
  'pred_scale': 'list',
  'em': 1.0,
  'f1': 1.0,
  'pred_span': None,
  'gold_span': None,
  'span_em': 0,
  'span_f1': 0},
 {'answer_type': 'multi-span',
  'answer': ['Transportation Solutions',
   'Industrial Solutions',
   'Communications Solutions'],
  'scale': '',
  'pred': ['Transportation Solutions',
   'Industrial Solutions',
   'Communications Solutions'],
  'pred_scale': 'list',
  'em': 0.0,
  'f1

In [ ]:
import re 
def remove_thousand_separators(text):
    # This regular expression matches numbers with commas as thousand separators
    return re.sub(r'(?<=\d),(?=\d{3})', '', text)
    
def transform_elements(list_of_lists, transform_func):    
    return [
        [transform_func(item) for item in sublist]
        for sublist in list_of_lists
    ]
    
def replace_bracketed_numbers(text):
    return re.sub(r"\((\d+)\)", lambda x: f"-{x.group(1)}", text)

In [ ]:
len(res)

In [1]:
good = 0
for ans, pred, pred_scale, q,code, _,table,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    
    if pred  is None or isinstance(pred, tuple) or isinstance(pred, str):
        #type error
        #if isinstance(pred, str):
        #    print (pred, pred_scale, ' -- ' , ans['answer'])
        #if isinstance(pred, tuple):
        #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        #print('===============================================================================')
        print(err)
        continue
    
    pred = round(pred, 2)
    
    if (pred > 0 and llimit < pred and pred < ulimit) or (pred < 0 and llimit > pred and pred > ulimit) or pred==ulimit or pred == llimit:
        good = good + 1
        #if  ans['scale'] == 'thousand':
        #    print (q['uid'], pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
          
    else:
        #value error
        print (q['uid'], pred, pred_scale, ' <> ' , ans['answer'], ans['scale'])
        #print(q['question'])
        #print(table['table'])
        #print(code)
        
    #if isinstance(pred, str):
    #    print (pred, pred_scale, ' -- ' , ans['answer'])
    #if isinstance(pred, tuple):
    #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        
print (good/len(res))

NameError: name 'res' is not defined

In [23]:
res


[({'answer_type': 'span',
   'answer': ['our allowable incurred costs plus a profit which can be fixed or variable depending on the contract’s fee arrangement up to predetermined funding levels determined by the customer'],
   'scale': ''},
  'r',
  None,
  {'uid': '23801627-ff77-4597-8d24-1c99e2452082',
   'order': 1,
   'question': 'What is the company paid on a cost-plus type contract?',
   'answer': ['our allowable incurred costs plus a profit which can be fixed or variable depending on the contract’s fee arrangement up to predetermined funding levels determined by the customer'],
   'derivation': '',
   'answer_type': 'span',
   'answer_from': 'text',
   'rel_paragraphs': ['2'],
   'req_comparison': False,
   'scale': ''},
  '',
  'Find the answer to the following question in the following texts or table! Return the found text spans as Python list. Try to delete question from result.  If the answer is not in the text or table return \'None\'. \n Question: {\'uid\': \'23801627-ff77